# Install

In [ ]:
# !pip install neo4j-driver
# !pip install gradio

# Imports

In [62]:
from neo4j import GraphDatabase
import gradio as gr
import re

# Connect to Neo4J

In [218]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", 
                              auth=("neo4j", "password"))

# Functions

In [219]:
def extract_title(question, type):
    # Define the regex pattern to match the title
    if type == "cast":
        pattern = r"(?i)siapa(?:kah)? pemain(?:\s+\w+)* (?:drama|series) (.+?)[\?\.]"
    elif type == "genre" :
        pattern = r"(?i)series\s+(.+?)\s+memiliki\s+genre"
    elif type == "director" :
        pattern = r"(?i)siapa(?:kah)? sutradara(?:\s+\w+)* (?:drama|series) (.+?)[\?\.]"
    elif type == "citizen" :
        pattern = r"(?i)apa warga negara(?:\s+\w+)* pemain (?:drama|series) (.+?)[\?\.]"
    
    # Search for the pattern in the question
    match = re.search(pattern, question, re.IGNORECASE)
    
    # If a match is found, return the matched group (title)
    if match:
        return match.group(1).strip()
    else:
        return None
    
def get_number(question):
    # Define the regex pattern to match numbers
    pattern = r'\b\d+\b'  # Matches one or more digits surrounded by word boundaries

    # Search for the pattern in the input string
    matches = re.findall(pattern, question)

    # Extract the first match as X value
    if matches:
        X = int(matches[0])
        return X
    else:
        print("No number found in the input string.")
        return None

In [220]:
def get_query(find_keys, question):
    # Siapakah pemain dalam series X?
    if all(key in find_keys for key in ["siapa", "pemain"]):
        title = extract_title(question, "cast")
        print(title)
        if title != None:
         return(f"MATCH (s)-[:HAS_CAST]->(c) WHERE s.name = '{title}' RETURN c.label AS cast", ["cast"])
    
    #Series X memiliki genre apa?
    elif all(key in find_keys for key in ["genre"]):
        title = extract_title(question, "genre")
        print(title)
        if title != None:
         return(f"MATCH (s)-[:HAS_GENRE]->(g) WHERE s.name = '{title}' RETURN g.label AS genre", ["genre"])
        
    #Siapa sutradara (director) dari series X?
    elif all(key in find_keys for key in ["siapa", "sutradara"]):
        title = extract_title(question, "director")
        print(title)
        if title != None:
         return(f"MATCH (s)-[:HAS_DIRECTOR]->(d) WHERE s.name = '{title}' RETURN d.label AS director_name", ["director_name"])
        
    # Apa warga negara dari pemain series X?
    elif all(key in find_keys for key in ["pemain", "warga negara"]):
        title = extract_title(question, "citizen")
        print(title)
        if title != None:
         return(f"MATCH (s)-[:HAS_CAST]->(c)-[CITIZENSHIP]->(cs) WHERE s.name = '{title}' RETURN c.label AS cast, cs.label AS warga_negara", ["cast", "warga_negara"])
       
    # Series yang memiliki jumlah episode lebih dari/kurang dari/(sama dengan) X?
    elif all(key in find_keys for key in ["episode"]):
       num_ep = get_number(question)
       if all(key in find_keys for key in ["lebih"]):
          return(f"MATCH (s) WHERE s.NUM_OF_EPS > {num_ep} RETURN s.name AS series", ["series"])
       elif all(key in find_keys for key in ["kurang"]):
          return(f"MATCH (s) WHERE s.NUM_OF_EPS < {num_ep} RETURN s.name AS series", ["series"])
       else:
          return(f"MATCH (s) WHERE s.NUM_OF_EPS = {num_ep} RETURN s.name AS series", ["series"])
    print("no matching!")
    return None

In [221]:
def get_answer(question):
    # cari kata kunci
    keys = ["siapa", "pemain", "genre", "sutradara", "warga negara", "episode", "kurang", "lebih"]
    find_keys = []
    for key in keys:
        # print(key)
        if key in question.lower() :
             find_keys.append(key)
         

    print(find_keys)
    # query yang terpilih berdasarkan kata kunci
    query, out = get_query(find_keys, question)
    
    # run query
    if query != None:
        with driver.session(database="tvseries") as session:
            result = session.run(query)
            # print(result[0])
            str = ""
            for record in result:
                    for label in out:
                        str += f"{label}: {record[label]} \n"
            return str
    else: 
         print("Not Recognize")


In [222]:
get_answer("Series yang memiliki jumlah episode lebih dari 100")

['episode', 'lebih']


"series: The Game \nseries: Whose Line Is It Anyway? \nseries: Saturday Night Live \nseries: Hotel 13 \nseries: The Sullivans \nseries: Nashville \nseries: The Brady Bunch \nseries: Pretty Little Liars \nseries: I Love Lucy \nseries: The Bold and the Beautiful \nseries: Melissa & Joey \nseries: ER \nseries: Everybody Loves Raymond \nseries: Psych \nseries: M jak miłość \nseries: The X-Files \nseries: Grimm \nseries: Afterworld \nseries: The Waltons \nseries: Gunsmoke \nseries: That '70s Show \nseries: Walker, Texas Ranger \nseries: Angel \nseries: Glukhar \nseries: Will & Grace \nseries: Na Wspólnej \nseries: La que se avecina \nseries: Magnum, P.I. \nseries: Wizards of Waverly Place \nseries: Sasural Simar Ka \nseries: Days of Our Lives \nseries: Brothers & Sisters \nseries: New Tricks \nseries: Chespirito \nseries: Late Night with David Letterman \nseries: Shadow Warriors \nseries: The Big Bang Theory \nseries: Klan \nseries: Yo! MTV Raps \n"

# Interface

In [121]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history

In [122]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


siapa
pemain
['siapa', 'pemain']
B
siapa
pemain
['siapa', 'pemain']
B
siapa
pemain
['siapa', 'pemain']
B
Keyboard interruption in main thread... closing server.


In [217]:
driver.close()